# RNN
## hello를 예로 들자면, h가 나오면 다음에 e가 나올것을 예측
## 이런 모델을 Character Set Language Model 이라고 함.

In [43]:
import numpy as np
import pprint
import tensorflow as tf
from tensorflow.contrib import rnn
pp = pprint.PrettyPrinter( indent = 4 )
sess = tf.InteractiveSession()

C:\Python\Python37\lib\site-packages\tensorflow\python\client\session.py:1735: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [44]:
h = [1, 0, 0, 0]
e = [0, 1, 0, 0]
l = [0, 0, 1, 0]
o = [0, 0, 0, 1]

In [45]:
# Cell 만드는 작업
# hidden_size가 결정이 되야 output_size가 결정이 됨.
with tf.variable_scope('one_cell') as scope:
    hidden_size = 2
    cell = tf.keras.layers.SimpleRNNCell(units = hidden_size)
    print(cell.output_size, cell.state_size)
    
    x_data = np.array([[h]], dtype = np.float32)
    pp.pprint(x_data)
    output, _states = tf.nn.dynamic_rnn(cell, x_data, dtype = tf.float32)
    
    sess.run(tf.global_variables_initializer())
    pp.pprint(output.eval())

2 2
array([[[1., 0., 0., 0.]]], dtype=float32)
array([[[0.43940946, 0.64147514]]], dtype=float32)


In [46]:
with tf.variable_scope('two_sequences') as scope:
    hidden_size = 2
    cell = tf.keras.layers.SimpleRNNCell(units = hidden_size)
    x_data = np.array([[h, e, l, l, o]], dtype = np.float32)
    print(x_data.shape)
    pp.pprint(x_data)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype = tf.float32)
    
    sess.run(tf.global_variables_initializer())
    pp.pprint(output.eval())
    
#(1, 5, 4) =  1 : list 개수,  5 : Sequence 개수, 4 : Sequence Length

(1, 5, 4)
array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]]], dtype=float32)
array([[[-0.6803115 , -0.20076342]]], dtype=float32)


In [47]:
with tf.variable_scope('3_batches') as scope:
    x_data = np.array([[h, e, l, l, o],
                       [e, o, l, l, l],
                       [l, l, e, e, l]], dtype = np.float32)
    pp.pprint(x_data)
    
    hidden_size = 2
    cell = tf.nn.rnn_cell.LSTMCell(num_units = hidden_size, state_is_tuple = True)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, dtype = tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(outputs.eval())

array([[[1., 0., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 0., 1.]],

       [[0., 1., 0., 0.],
        [0., 0., 0., 1.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 0., 1., 0.]],

       [[0., 0., 1., 0.],
        [0., 0., 1., 0.],
        [0., 1., 0., 0.],
        [0., 1., 0., 0.],
        [0., 0., 1., 0.]]], dtype=float32)


ValueError: Variable 3_batches/rnn/lstm_cell/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "C:\Python\Python37\lib\site-packages\tensorflow\python\framework\ops.py", line 2005, in __init__
    self._traceback = tf_stack.extract_stack()
  File "C:\Python\Python37\lib\site-packages\tensorflow\python\framework\ops.py", line 3616, in create_op
    op_def=op_def)
  File "C:\Python\Python37\lib\site-packages\tensorflow\python\util\deprecation.py", line 507, in new_func
    return func(*args, **kwargs)
  File "C:\Python\Python37\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 788, in _apply_op_helper
    op_def=op_def)
  File "C:\Python\Python37\lib\site-packages\tensorflow\python\ops\gen_state_ops.py", line 2023, in variable_v2
    shared_name=shared_name, name=name)


In [48]:
batch_size = 3
sequence_length = 5
input_dim = 3
x_data = np.arange(45, dtype = np.float32).reshape(batch_size, sequence_length, input_dim)
pp.pprint(x_data)

array([[[ 0.,  1.,  2.],
        [ 3.,  4.,  5.],
        [ 6.,  7.,  8.],
        [ 9., 10., 11.],
        [12., 13., 14.]],

       [[15., 16., 17.],
        [18., 19., 20.],
        [21., 22., 23.],
        [24., 25., 26.],
        [27., 28., 29.]],

       [[30., 31., 32.],
        [33., 34., 35.],
        [36., 37., 38.],
        [39., 40., 41.],
        [42., 43., 44.]]], dtype=float32)


In [49]:
with tf.variable_scope('generated_data') as scope:
    cell = tf.nn.rnn_cell.LSTMCell(num_units = 5, state_is_tuple = True)
    initial_state = cell.zero_state( batch_size, tf.float32)
    outputs, _states = tf.nn.dynamic_rnn(cell, x_data, initial_state = initial_state, dtype = tf.float32)
    sess.run(tf.global_variables_initializer())
    pp.pprint(output.eval())

ValueError: Variable generated_data/rnn/lstm_cell/kernel already exists, disallowed. Did you mean to set reuse=True or reuse=tf.AUTO_REUSE in VarScope? Originally defined at:

  File "C:\Python\Python37\lib\site-packages\tensorflow\python\framework\ops.py", line 2005, in __init__
    self._traceback = tf_stack.extract_stack()
  File "C:\Python\Python37\lib\site-packages\tensorflow\python\framework\ops.py", line 3616, in create_op
    op_def=op_def)
  File "C:\Python\Python37\lib\site-packages\tensorflow\python\util\deprecation.py", line 507, in new_func
    return func(*args, **kwargs)
  File "C:\Python\Python37\lib\site-packages\tensorflow\python\framework\op_def_library.py", line 788, in _apply_op_helper
    op_def=op_def)
  File "C:\Python\Python37\lib\site-packages\tensorflow\python\ops\gen_state_ops.py", line 2023, in variable_v2
    shared_name=shared_name, name=name)


In [54]:
#RNN으로 학습하고, Loss가 어떤지 확인을 해야함.
#작을수록 학습이 잘된 것.
y_data = tf.constant([[1, 1, 1]])

prediction1 = tf.constant([[[0.3, 0.7], [0.3, 0.7], [0.3, 0.7]]],
                         dtype = tf.float32)
prediction2 = tf.constant([[[0.1, 0.9], [0.1, 0.9], [0.1, 0.9]]],
                         dtype = tf.float32)
prediction3 = tf.constant([[[1, 0], [1, 0], [1, 0]]],
                         dtype = tf.float32)
prediction4 = tf.constant([[[0, 1], [0, 1], [0, 1]]],
                         dtype = tf.float32)

weights = tf.constant([[1, 1, 1]], dtype = tf.float32)
sequence_loss1 = tf.contrib.seq2seq.sequence_loss(prediction1, y_data, weights)
sequence_loss2 = tf.contrib.seq2seq.sequence_loss(prediction2, y_data, weights)
sequence_loss3 = tf.contrib.seq2seq.sequence_loss(prediction3, y_data, weights)
sequence_loss4 = tf.contrib.seq2seq.sequence_loss(prediction4, y_data, weights)
sess.run(tf.global_variables_initializer())
print('Loss1 : ', sequence_loss1.eval(), '\nLoss2 : ', sequence_loss2.eval(),
      '\nLoss3 : ', sequence_loss2.eval(), '\nLoss4 : ', sequence_loss4.eval())


Loss1 :  0.5130153 
Loss2 :  0.3711007 
Loss3 :  0.3711007 
Loss4 :  0.31326166


In [55]:
#RNN
tf.set_random_seed(777) # reproducibility

idx2char = ['h', 'i', 'e', 'l', 'o']

SyntaxError: invalid syntax (<ipython-input-55-6e8b56e8e475>, line 2)